# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [1]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [2]:
#Load Data
path = '../data/Subset 1/'
subdirs = glob.glob('%s*'%(path))


data = None
temp = []
flag = 0
for dir in subdirs:
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir)
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

/home/abhijit/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [19]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Tag
        try:
            tag = str(entry[1]['Tag']).split('-')[0].strip()
        except Exception as e:
            print(e,'####')
        try:
            stag = str(entry[1]['StudentTag']).split('-')[0].strip()
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [20]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], 'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],'StudentTag':[i[4] for i in dataset],'Transcript':[i[5] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df

,Turn,Speaker,Sentence,Tag,StudentTag,Transcript
0,1,T,This goes under your desk Jared,1.0,nan,Math US2 transcript.xlsx
1,2,S,Bless you,nan,1,Math US2 transcript.xlsx
2,3,T,Okay everybody please put your books away,1.0,nan,Math US2 transcript.xlsx
3,3,T,Quietly take out paper to start the warmup,1.0,nan,Math US2 transcript.xlsx
4,4,T,Quietly have your homework out on your desk,1.0,nan,Math US2 transcript.xlsx
...,...,...,...,...,...,...
57603,581,T,What you did and why,1.0,nan,Ribbons 2_Grade 4.xlsx
57604,582,S,So whatever answer you did,1.0,1,Ribbons 2_Grade 4.xlsx
57605,582,S,Whatever answers you did,1.0,1,Ribbons 2_Grade 4.xlsx
57606,583,T,And how,1.0,nan,Ribbons 2_Grade 4.xlsx


In [7]:
writer = pd.ExcelWriter('../data/subset1.xlsx')
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [9]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/subset1.xlsx')
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})

df.to_csv('../data/susbet1_student.tsv',sep='\t',index=False)       

In [10]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/subset1.xlsx')
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})

df.to_csv('../data/susbet1_teacher.tsv',sep='\t',index=False)    